**Домашнее задание**

Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/  
Реализовать самостоятельно логистическую регрессию  
Обучить ее методом градиентного спуска  
Методом nesterov momentum  
Методом rmsprop  
В качестве dataset'а взять Iris, оставив 2 класса:  
Iris Versicolor  
Iris Virginica  

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
ds = datasets.load_iris()
irises = pd.DataFrame(ds.data, columns=ds.feature_names)
irises["iris"] = [ds.target_names[x] for x in ds.target]
irises_selected = irises[irises["iris"].isin(["versicolor", "virginica"])].reset_index(drop=True)
irises_selected

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris
0,7.0,3.2,4.7,1.4,versicolor
1,6.4,3.2,4.5,1.5,versicolor
2,6.9,3.1,4.9,1.5,versicolor
3,5.5,2.3,4.0,1.3,versicolor
4,6.5,2.8,4.6,1.5,versicolor
...,...,...,...,...,...
95,6.7,3.0,5.2,2.3,virginica
96,6.3,2.5,5.0,1.9,virginica
97,6.5,3.0,5.2,2.0,virginica
98,6.2,3.4,5.4,2.3,virginica


In [3]:
print("Формируем набор исходных данных.")
X = irises_selected.copy()
del X["iris"]
X.head()

Формируем набор исходных данных.


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,7.0,3.2,4.7,1.4
1,6.4,3.2,4.5,1.5
2,6.9,3.1,4.9,1.5
3,5.5,2.3,4.0,1.3
4,6.5,2.8,4.6,1.5


In [4]:
print("Формируем набор результата.")
iris_en = LabelEncoder()
iris_en.fit(irises_selected["iris"])
print(f"Нумеруемые энкодером значения: {iris_en.classes_}.")
Y = pd.Series(iris_en.transform(irises_selected["iris"]))
Y.head()

Формируем набор результата.
Нумеруемые энкодером значения: ['versicolor' 'virginica'].


0    0
1    0
2    0
3    0
4    0
dtype: int64

In [5]:
print("Делим данные на выборки.")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

Делим данные на выборки.


In [6]:
def prepare_data(x, y):
    x_res = np.hstack([np.ones((x.shape[0], 1)), x]) # insert a column of ones for w0
    y_res = y.to_numpy()
    return x_res, y_res

def get_predict_proba(x, w): # z = w0 * 1 + w1*x1 + w2*x2 + ... + wn*xn, f(z) = 1/(1+e^(-z)) for each sample
    return 1.0 / (1 + np.exp(-np.dot(x, w)))

def get_predict(x, w):
    return np.round(get_predict_proba(x, w)).astype(int)

def get_change_size(value):
    return np.sum(np.abs(value))

def gradient_descent(x, y, iterations, learning_rate, treshold):
    error = None
    found = None
    m, n = x.shape  # rows, cols + 1
    w = np.zeros(n) # init the decision vector with zeros
    for i in range(iterations):
        h = get_predict_proba(x, w)
        grad = np.dot(x.T, h - y) / m
        v = learning_rate * grad
        w += -v
        if get_change_size(v) < treshold:
            found = i + 1
            error = - (np.dot(y, np.log(h)) + np.dot((1 - y), np.log(1 - h))) / m
            break
            
    return found, error, w

def gradient_descent_Nesterov(x, y, iterations, learning_rate, gamma, treshold):
    error = None
    found = None
    m, n = x.shape  # rows, cols + 1
    w = np.zeros(n) # init the decision vector with zeros
    prev = np.zeros(n)
    for i in range(iterations):
        v = gamma * prev
        h = get_predict_proba(x, w - v)
        grad = np.dot(x.T, h - y) / m
        v += learning_rate * grad
        w += -v
        prev = v
        if get_change_size(v) < treshold:
            found = i + 1
            error = - (np.dot(y, np.log(h)) + np.dot((1 - y), np.log(1 - h))) / m
            break
            
    return found, error, w

def gradient_descent_rmsprop(x, y, iterations, learning_rate, gamma, treshold):
    error = None
    found = None
    m, n = x.shape  # rows, cols + 1
    w = np.zeros(n) # init the decision vector with zeros
    prev = np.zeros(n)
    ep = 1e-6
    for i in range(iterations):
        h = get_predict_proba(x, w)
        grad = np.dot(x.T, h - y) / m
        e = gamma * prev + (1 - gamma) * grad.dot(grad)
        v = learning_rate * grad / np.sqrt(e + ep)
        w += -v
        prev = e
        if get_change_size(v) < treshold:
            found = i + 1
            error = - (np.dot(y, np.log(h)) + np.dot((1 - y), np.log(1 - h))) / m
            break
            
    return found, error, w


def interpret_results(x_train_np, y_train_np, x_test_np, y_test_np, found, error, w):
    if found is None:
        print(f"Недостаточно итераций или слишком низкий порог изменения.")
        return
    
    print(f"Найдены коэффициенты = {w}.\nЗначение ошибки = {error} на {found} итерации.")

    y_pred = get_predict(x_train_np, w)
    accur = accuracy_score(y_train_np, y_pred)
    print(f"Качество модели на тренировочных данных: {accur}.")

    y_pred = get_predict(x_test_np, w)
    accur = accuracy_score(y_test_np, y_pred)
    print(f"Качество модели на тестовых данных: {accur}.")
    return


x_train_np, y_train_np = prepare_data(X_train, Y_train)
x_test_np, y_test_np = prepare_data(X_test, Y_test)

iterations = 500
learning_rate = 0.9
treshold = 0.1
print("\nГрадиентный спуск")
found, error, w = gradient_descent(x_train_np, y_train_np, iterations, learning_rate, treshold)
interpret_results(x_train_np, y_train_np, x_test_np, y_test_np, found, error, w)

gamma = 0.9
iterations = 500
learning_rate = 0.9
treshold = 0.1
print("\nГрадиентный спуск (Nesterov)")
found, error, w = gradient_descent_Nesterov(x_train_np, y_train_np, iterations, learning_rate, gamma, treshold)
interpret_results(x_train_np, y_train_np, x_test_np, y_test_np, found, error, w)

gamma = 0.9992 #0.9
iterations = 500
learning_rate = 0.9
treshold = 0.1
print("\nГрадиентный спуск (RMSProp)")
found, error, w = gradient_descent_rmsprop(x_train_np, y_train_np, iterations, learning_rate, gamma, treshold)
interpret_results(x_train_np, y_train_np, x_test_np, y_test_np, found, error, w)

print("\nИнтересно, что RMSProp показал сопоставимый с Nesterov результат только с увеличением 𝛾, т.е. при уменьшении доли суммы квадратов обновления g^2 в накоплении среднего E.")
print("Т.е. снижая степень влияния улучшений RMSProp на немодифицированный вариант градиентного спуска.")
print("Хотя, наверное, в этом и смысл гиперпараметров.")


Градиентный спуск
Найдены коэффициенты = [-4.37043288 -8.9701663  -7.80579091 13.05778841 11.21476996].
Значение ошибки = 0.03690215085544598 на 152 итерации.
Качество модели на тренировочных данных: 0.9857142857142858.
Качество модели на тестовых данных: 0.9333333333333333.

Градиентный спуск (Nesterov)
Найдены коэффициенты = [ -7.35233184 -14.11389662 -13.45071208  21.2734809   17.58448922].
Значение ошибки = 0.02182009056005125 на 58 итерации.
Качество модели на тренировочных данных: 0.9857142857142858.
Качество модели на тестовых данных: 0.9333333333333333.

Градиентный спуск (RMSProp)
Найдены коэффициенты = [ -5.74500068 -12.17993019 -10.47246015  17.63137076  15.05958091].
Значение ошибки = 0.031162882882007192 на 66 итерации.
Качество модели на тренировочных данных: 0.9857142857142858.
Качество модели на тестовых данных: 0.9333333333333333.

Интересно, что RMSProp показал сопоставимый с Nesterov результат только с увеличением 𝛾, т.е. при уменьшении доли суммы квадратов обновлен